<a href="https://colab.research.google.com/github/yoouza/bigcontest/blob/master/data_preprocessing/News_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
import json, os, re

from tqdm import tqdm
from datetime import datetime
from nltk.tokenize import sent_tokenize

# **데이터 불러오기**

In [3]:
path = '/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/00. Data/07. 뉴스'
files = os.listdir(path)

news = []
for file in tqdm(files):
    with open(path+'/'+file, 'rb') as f:
        for item in json.load(f):
            news_content = {'date':0, 'title':0, 'text':0}
            news_content['date'] = item['date']
            news_content['title'] = item['title']
            news_content['text'] = item['text']
            news.append(news_content)

100%|██████████| 5/5 [00:58<00:00, 11.69s/it]


In [4]:
news_df = pd.DataFrame(news)
news_df.head(), len(news_df)

(         date  ...                                               text
 0  2019-02-01  ...  EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중  (브뤼셀·서울=연합뉴스)...
 1  2019-02-01  ...  [세종=이데일리 김형욱 기자] ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승...
 2  2019-02-01  ...  (뉴욕=연합뉴스) 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증...
 3  2019-02-01  ...  방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것”      ...
 4  2019-02-01  ...  [세종=이데일리 김형욱 기자] ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상...
 
 [5 rows x 3 columns], 605268)

# **데이터 전처리**

In [5]:
news_df = news_df.drop_duplicates(['text']) # 본문 중복 제거
news_df = news_df.reset_index()
news_df = news_df.drop(['index'], axis=1)
len(news_df)

372384

In [6]:
news_df = news_df.fillna("")
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 (브뤼셀·서울=연합뉴스)...
1,2019-02-01,[인사]산업통상자원부,[세종=이데일리 김형욱 기자] ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,(뉴욕=연합뉴스) 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증...
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” ...
4,2019-02-01,[인사]한국로봇산업진흥원,[세종=이데일리 김형욱 기자] ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상...


In [7]:
news_df['text'] = news_df.text.apply(lambda x : x.replace('\\t', '').replace('\n', '').replace('  ', ' ').replace('   ', ' '))
news_df['text'] = news_df.text.apply(lambda x : re.sub('[ A-Za-z0-9]*\@[a-z]*\.[a-z]*.kr', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('[ A-Za-z0-9]*\@[a-z]*\.[a-z]*', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\[[가-히a-zA-Z=·, @….0-9/:]*\]', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\([가-히a-zA-Z=·, @….0-9/:]*\)', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\【[가-히a-zA-Z=·, @….0-9/:]*\】', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\<[가-히a-zA-Z=·, @….]*\>', '', x))
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 기...
1,2019-02-01,[인사]산업통상자원부,◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승진 △장관정책보좌관 김완기 ◇...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다. ...
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” 김현종 ...
4,2019-02-01,[인사]한국로봇산업진흥원,◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상미 △시설안전보안팀장 김덕구 ...


In [8]:
news_df['text'] = news_df.text.apply(lambda x : sent_tokenize(x))
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 ...
1,2019-02-01,[인사]산업통상자원부,[ ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승진 △장관정책보좌관 김완기 ...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,[ 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다....
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",[방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” 김현종...
4,2019-02-01,[인사]한국로봇산업진흥원,[ ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상미 △시설안전보안팀장 김덕구...


# **데이터 라벨링**

In [9]:
news_df['new_date'] = 0
news_df['new_date'] = news_df.date.apply(lambda x : str(datetime.strptime(x, "%Y-%m-%d"))[:10])
news_df.head()

,date,title,text,new_date
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 ...,2019-02-01
1,2019-02-01,[인사]산업통상자원부,[ ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승진 △장관정책보좌관 김완기 ...,2019-02-01
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,[ 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다....,2019-02-01
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",[방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” 김현종...,2019-02-01
4,2019-02-01,[인사]한국로봇산업진흥원,[ ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상미 △시설안전보안팀장 김덕구...,2019-02-01


In [10]:
all_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/code_rf_2019_ud_rolling1.csv'
all_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/code_rf_2020_ud_rolling1.csv'

In [11]:
all_2019_df = pd.read_csv(all_2019_path, index_col=0)
all_2020_df = pd.read_csv(all_2020_path, index_col=0)
all_2019_df.head(), all_2020_df.head()

(         date     USE_AMT    1d_later  ud
 0  2019-02-01  27745677.0  27587295.0  10
 1  2019-02-02  32647543.0  32679856.0  10
 2  2019-02-03  27280036.0  26983048.0  10
 3  2019-02-04  22093001.0  22006479.0  10
 4  2019-02-05  12040355.0  22761211.0   1,
          date     USE_AMT    1d_later  ud
 0  2020-02-01  26166362.0  23263083.0   0
 1  2020-02-02  20420944.0  18834301.0   0
 2  2020-02-03  17182754.0  17007465.0  10
 3  2020-02-04  17957526.0  19621540.0   1
 4  2020-02-05  18126539.0  18539953.0  10)

In [12]:
def attach_label(df1, df2, news_df):
    total_df = pd.concat([df1, df2])
    df1 = df1.reset_index()
    # 날짜 데이트화
    df1['new_date'] = str(0)
    for i in range(len(df1)):
        df1['new_date'][i] = str(datetime.strptime(df1['date'][i], "%Y-%m-%d"))[:10]
    # 라벨만 달린 날짜
    diff = df1[['new_date', 'ud']]
    # 레프트 조인
    news_wt_diff = pd.merge(diff, news_df, how = 'left', on='new_date')
    news_wt_diff = news_wt_diff.sort_values(by = ['date'], axis=0, ascending=True)
    news_wt_diff = news_wt_diff.reset_index()
    news_wt_diff = news_wt_diff.drop(['index'], axis=1)
    # 필요한 데이터만 가져오기
    data = news_wt_diff[['new_date', 'text', 'ud']]
    data = data.dropna()
    data = data.reset_index()
    # 뉴스 한 줄씩 라벨링
    new_data = []
    for i in tqdm(range(len(data))):
        for j in range(len(data['text'][i])-1): # 마지막줄 제거
            tmp_data = {'date' : 0, 'content' : 0, 'label' : 0}
            tmp_data['date'] = data['new_date'][i]
            if len(data['text'][i][j]) < 10:
                tmp_data['content'] = 0
            else:
                tmp_data['content'] = data['text'][i][j]
            tmp_data['label'] = data['ud'][i]
            new_data.append(tmp_data)
    
    return pd.DataFrame(new_data)

In [13]:
data_all = attach_label(all_2019_df, all_2020_df, news_df)
data_all.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 111032/111032 [01:31<00:00, 1215.51it/s]


,date,content,label
0,2019-02-01,EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 기...,10
1,2019-02-01,병든 소를 도축하는 폴란드 도축장 영상 유럽연합 행정부 격인 집행위원회는 1일 폴...,10
2,2019-02-01,EU 집행위 대변인은 이날 브리핑에서 폴란드 당국이 소고기 가공공장을 폐쇄하고 시장...,10
3,2019-02-01,"문제의 소고기가 수출된 것으로 확인된 나라는 에스토니아, 핀란드, 프랑스, 독일, ...",10
4,2019-02-01,앞서 영국 BBC 방송은 폴란드의 한 도축장에서 도축된 병든 소가 프랑스와 독일 등...,10


In [14]:
set(data_all['label'].to_list())

{0, 1, 10}

In [15]:
len(data_all)

1073289

In [16]:
data_all = data_all[data_all['content']!=0]
data_all = data_all.reset_index()
data_all = data_all.drop(['index'], axis=1)
data_all.head()

,date,content,label
0,2019-02-01,EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 기...,10
1,2019-02-01,병든 소를 도축하는 폴란드 도축장 영상 유럽연합 행정부 격인 집행위원회는 1일 폴...,10
2,2019-02-01,EU 집행위 대변인은 이날 브리핑에서 폴란드 당국이 소고기 가공공장을 폐쇄하고 시장...,10
3,2019-02-01,"문제의 소고기가 수출된 것으로 확인된 나라는 에스토니아, 핀란드, 프랑스, 독일, ...",10
4,2019-02-01,앞서 영국 BBC 방송은 폴란드의 한 도축장에서 도축된 병든 소가 프랑스와 독일 등...,10


# **데이터 저장**

In [17]:
data_all.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_data.csv')